In [26]:
# pip install requests
import requests
# pip install parsel 
from parsel import Selector

In [27]:
faqEntries = {}
def extractEntries(currentPage):
    currentEntries = currentPage.css('span a[href]')
    for e in currentEntries:
        link = e.xpath('@href').get()
        question = e.xpath('text()').get()
        faqEntries[link] = question


Load initial page

In [28]:
response = requests.get('https://www.faq.bag.admin.ch/en/categories/vaccination')
selector = Selector(response.text)

Load questions of initial page

In [29]:
extractEntries(selector)

Load other available pages

In [32]:
faqPageSelectors = selector.css('a[title][href]')
faqPageUrls = []
for p in faqPageSelectors:
    url = p.xpath('@href').get()
    if "page" in url:
        response = requests.get('https://www.faq.bag.admin.ch'+url)
        selector = Selector(response.text)
        extractEntries(selector)

Count available questions

In [34]:
len(faqEntries)

60

Extract answers

In [36]:
class FAQEntry:
    link = ''
    question = ''
    answer = ''

finalEntries = []
for e in faqEntries:
    response = requests.get('https://www.faq.bag.admin.ch'+e)
    selector = Selector(response.text)
    entry = FAQEntry()
    entry.link = e
    entry.question = faqEntries[e]
    entry.answer = selector.css('.field-item.even').get()
    finalEntries.append(entry)
finalEntries